## import used libraries

In [1]:
import cv2
import os
import json
import random

## Handy function for showing images

In [2]:
def show(file, img):
    cv2.imshow(file, img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


## Get images and initialze result with empty array

In [23]:
path = "./data"
files = [f for f in os.listdir(path) if f.endswith(".png")]
print(files)
results = []


['10070.png', '1004.png', '10047.png', '10032.png', '10021.png', '10048.png', '10068.png', '10038.png', '10071.png', '10007.png', '10039.png', '10008.png', '10027.png', '10000.png', '10066.png', '10063.png', '10006.png', '10013.png', '10062.png', '10057.png', '1001.png', '10030.png', '10067.png', '10022.png', '10035.png', '10045.png', '10055.png', '10065.png', '10019.png', '10043.png', '1005.png', '10017.png', '10034.png', '10004.png', '10033.png', '1002.png', '10041.png', '10046.png', '10037.png', '1003.png', '10011.png', '1000.png', '10051.png', '10053.png', '10002.png', '10003.png', '10020.png', '10058.png', '10042.png', '10072.png', '10050.png', '10059.png', '10054.png', '10016.png', '1006.png', '10024.png', '10001.png', '10029.png', '10009.png', '10025.png', '10044.png', '10031.png', '10005.png', '10036.png', '10028.png', '10064.png', '10012.png', '10056.png', '10014.png', '10060.png', '10010.png', '10023.png', '10052.png', '10015.png', '10061.png', '10026.png', '10049.png', '1006

## Define the Algorithm to detect digits

In [18]:
def detect_digits(file):
    # Load the image and convert to grayscale
    img = cv2.imread(file)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Increase image contrast using histogram equalization
    gray = cv2.equalizeHist(gray)

    # Sharpen the image using the unsharp masking technique
    blurred = cv2.GaussianBlur(gray, (0, 0), 3)
    sharpened = cv2.addWeighted(gray, 1.5, blurred, -0.5, 0)
    
    # Apply adaptive thresholding to the image
    binary = cv2.adaptiveThreshold(
        sharpened, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, 11, 2)
    inverted = 255 - binary
    
        # Apply Canny edge detection to the binary image
    edges = cv2.Canny(binary, 100, 200)

    # Find contours of the digits
    contours, _ = cv2.findContours(
        binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours_inv, _ = cv2.findContours(
        inverted, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
       # Filter out small contours
    min_contour_area = 50
    contours = [c for c in contours if cv2.contourArea(
        c) > min_contour_area] + [c for c in contours_inv if cv2.contourArea(c) > min_contour_area]
    
        # Define minimum and maximum aspect ratio for digits
    min_aspect_ratio = 0.2
    max_aspect_ratio = 2.5
     # Draw bounding boxes around the regions that are likely to contain digits
    for contour in contours:
        left, top, width, height = cv2.boundingRect(contour)

        aspect_ratio = width / float(height)

        # Filter out regions with aspect ratio outside the desired range
        if aspect_ratio < min_aspect_ratio or aspect_ratio > max_aspect_ratio:
            continue

        std_dev_threshold = 15
        std_dev = cv2.meanStdDev(
            gray[top:top+height, left:left+width])[1][0][0]
        if std_dev < std_dev_threshold:
            continue
            

        # Calculate image dimensions
        img_height, img_width = img.shape[:2]
        
        # The digits is likely to be in the middle of the image so filter out contours on the far edges
        if (left < int(0.1 * img_width)) or ((left + width) > int(0.9 * img_width)):
            continue

        # Define minimum and maximum dimensions for digits
        minWidth = int(0.04 * img_width)
        minHeight = int(0.04 * img_height)
        maxWidth = int(0.27 * img_width)
        maxHeight = int(0.95 * img_height)
        

        if width < minWidth or height < minHeight or width > maxWidth or height > maxHeight:
            continue
            
        # Draw bounding boxes around the regions that are likely to contain digits
        cv2.rectangle(img, (left, top), (left + width,
                      top + height), (0, 0, 255), 1)

    # Display the image with bounding boxes around the digits
    show(file, img)

    

## Test algorithm accuracy

## Loop on some files to test the algorithms

In [25]:
for i in range(20):
    file = f'{path}/{files[i]}'
    print(file)
    boxes = []    
    detect_digits(file)
    
cv2.destroyAllWindows()



./data/10070.png
./data/1004.png
./data/10047.png
./data/10032.png
./data/10021.png
./data/10048.png
./data/10068.png
./data/10038.png
./data/10071.png
./data/10007.png
./data/10039.png
./data/10008.png
./data/10027.png
./data/10000.png
./data/10066.png
./data/10063.png
./data/10006.png
./data/10013.png
./data/10062.png
./data/10057.png
